# Linear Regression

## STILL IN THE PROCESS becuase I struggle with web srapping

This is the second version of the Linear regression for the IMBd movie dataset.

This version containts 2 datasets:
1 - original one
2 - more data from web scrapping

All steps will be repeated but in the beginning we will upload 2 datasets insted of one and combine them. 

It's important to note that the results may vary compared to the initial analysis. I will comment on the metrics and provide insights in the analysis section.

**Cleaning and modifying data**

In [38]:
# install all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [39]:
# uploading dataset to juputer notebook
df_1 = pd.read_csv("imdb_movie_data_2023.csv")
df_2 = pd.read_csv("New_data.csv")

In [56]:
print(df_1.shape)


(1950, 11)


In [58]:
column_counts = df_1.count()

print(column_counts)

Unnamed: 0    1950
Moive Name    1950
Rating        1944
Votes         1944
Meta Score    1833
Genre         1833
PG Rating     1874
Year          1950
Duration      1948
Cast          1833
Director      1833
dtype: int64


In [59]:
column_counts = df_2.count()

print(column_counts)

Unnamed: 0    1750
Movie Name    1750
Rating         255
Votes          255
Meta Score      58
Genre           58
PG Rating        0
Year             0
Duration         0
Cast            57
Director        58
dtype: int64


In [57]:
print(df_2.shape)

(1750, 11)


In [40]:
import pandas as pd

# Assuming df1 and df2 are your DataFrames
df = df_1.append(df_2, ignore_index=True)

/var/folders/lj/tcwl5glj0blfftjn4df0129r0000gn/T/ipykernel_90102/30573836.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_1.append(df_2, ignore_index=True)


In [41]:
df = pd.concat([df_1, df_2], ignore_index=True)

In [42]:
print(df.shape)


(3700, 12)


In [43]:
df.describe()

,Unnamed: 0,Rating,Votes,Meta Score,Year
count,3700.000000,2199.000000,2.199000e+03,1891.000000,1950.000000
mean,927.202703,6.847794,2.390762e+05,62.261237,2006.976410
std,538.775834,0.999483,3.077491e+05,16.537611,15.798125
min,0.000000,2.400000,6.000000e+00,14.000000,1938.000000
25%,462.000000,6.300000,2.800000e+04,51.000000,1999.000000
50%,924.500000,7.000000,1.470000e+05,63.000000,2011.000000
75%,1387.000000,7.500000,3.180000e+05,74.000000,2019.000000
max,1949.000000,9.800000,2.800000e+06,100.000000,2023.000000


In [44]:
column_counts = df.count()

print(column_counts)

Unnamed: 0    3700
Moive Name    1950
Rating        2199
Votes         2199
Meta Score    1891
Genre         1891
PG Rating     1874
Year          1950
Duration      1948
Cast          1890
Director      1891
Movie Name    1750
dtype: int64


In [45]:
# read the data and see how I can modify this dataset
df.head()

# I have to google some names of columns to better understand what they mean
# Metascore is considered the rating of a film. Scores are assigned 
# to movie's reviews of large group of the world's most respected critics, 
# and weighted average are applied to summarize their opinions range.
# https://www.imdb.com/list/ls051211184/#:~:text=Metascore%20is%20considered%20the%20rating,to%20summarize%20their%20opinions%20range.

# PG rating means to what audience these movies
# if there is any restrictions for audience

,Unnamed: 0,Moive Name,Rating,Votes,Meta Score,Genre,PG Rating,Year,Duration,Cast,Director,Movie Name
0,0,Leave the World Behind,6.5,90000.0,67.0,"Drama, Mystery, Thriller",R,2023.0,2h 18m,"Julia Roberts, Mahershala Ali, Ethan Hawke, My...",Sam Esmail,NaN
1,1,Wonka,7.4,24000.0,66.0,"Adventure, Comedy, Family",PG,2023.0,1h 56m,"Timothée Chalamet, Gustave Die, Murray McArthu...",Paul King,NaN
2,2,Poor Things,8.5,6700.0,86.0,"Comedy, Drama, Romance",R,2023.0,2h 21m,"Emma Stone, Mark Ruffalo, Willem Dafoe, Ramy Y...",Yorgos Lanthimos,NaN
3,3,Killers of the Flower Moon,7.8,128000.0,89.0,"Crime, Drama, History",R,2023.0,3h 26m,"Leonardo DiCaprio, Robert De Niro, Lily Gladst...",Martin Scorsese,NaN
4,4,May December,7.0,21000.0,85.0,"Comedy, Drama",R,2023.0,1h 57m,"Natalie Portman, Chris Tenzis, Charles Melton,...",Todd Haynes,NaN


In [46]:
# we drop all NaN values
# in order to proceed futher with all modifications
df.dropna(inplace=True)

In [47]:
# check the format of columns
df.dtypes

# I need to check and if it is possible to modify the next columns:
# Genre, PG Rating, Duration

Unnamed: 0      int64
Moive Name     object
Rating        float64
Votes         float64
Meta Score    float64
Genre          object
PG Rating      object
Year          float64
Duration       object
Cast           object
Director       object
Movie Name     object
dtype: object

In [48]:
# I can drop columns Cast and Director
# because they containt a lot of data that cannot be modified to numeric
# the Moive Name is unnecessary for the Linear Regression
# that is why we also drop this column
df = df.drop(columns=['Cast', 'Director', 'Moive Name'])

In [49]:
# Splitting strings in the Genres column
# based on a separator ","
# and creating one-hot encoding
genres = df['Genre'].str.get_dummies(sep=', ')

# Concatenate one-hot encoded genres with original DataFrame
df = pd.concat([df, genres], axis=1)

# Dropping the original 'Genre' column
df.drop('Genre', axis=1, inplace=True)

In [50]:
# check that we do not have any same names of columns
columns_list = df.columns.tolist()
sorted_columns = sorted(columns_list)
sorted_columns

['Duration',
 'Meta Score',
 'Movie Name',
 'PG Rating',
 'Rating',
 'Unnamed: 0',
 'Votes',
 'Year']

In [51]:
# checking that the conversion was correct
# by looking at genres and their marks in movies
df.T

""
Unnamed: 0
Rating
Votes
Meta Score
PG Rating
Year
Duration
Movie Name


In [52]:
# verifing the current state of the DataFrame
df.head()

,Unnamed: 0,Rating,Votes,Meta Score,PG Rating,Year,Duration,Movie Name


In [53]:
# I can drop "Unnamed: 0"
# it was created becuse of contact of 2 df
df = df.drop(columns=['Unnamed: 0'])

In [54]:
df.head()

,Rating,Votes,Meta Score,PG Rating,Year,Duration,Movie Name


In [55]:
# this makes multiple columns with the variable PG Rating
from sklearn.preprocessing import OneHotEncoder
variables = ['PG Rating']

# I use encoder in order to make columns with only numeric data
encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
one_hot_encoded = encoder.fit_transform(df[variables]).astype(int)
df = pd.concat([df,one_hot_encoded],axis=1).drop(columns=variables)

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [ ]:
# verifing the current state of the DataFrame
df

In [ ]:
# we can drop PG Rating_Unrated for the same reason
# and also we can delete the last column: PG Rating_X
# becuase we used OneHotEncoder
df = df.drop(columns=['PG Rating_Unrated', 'PG Rating_X'])

In [ ]:
# we can start to modify the last column Duration
# we need to remove "h" and "m" and modify the data it into minutes
# we can make a function that convert a string column
# into the numeric one
def convert_to_minutes(duration_str):
    try:
        # if the value is already an integer, return it as is
        if isinstance(duration_str, int):
            return duration_str

        # split the string into parts based on 'h' and 'm'
        parts = duration_str.split()

        # initialize hours and minutes
        hours, minutes = 0, 0

        # check each part and update hours or minutes accordingly
        for part in parts:
            if 'h' in part:
                hours = int(part.replace('h', ''))
            elif 'm' in part:
                minutes = int(part.replace('m', ''))

        # calculate total minutes
        total_minutes = hours * 60 + minutes
        return total_minutes
    except Exception as e:
        print(f"Error processing {duration_str}: {e}")
        return pd.NA

# appling the conversion function to the 'Duration' column
df['Duration'] = df['Duration'].apply(convert_to_minutes)

In [ ]:
# checking the dataset that we finally have
df.head()

# we can that all data that we have now is numeric
# and now we can proceed with checking the balance of the data

In [ ]:
# I change the view of the df in order to check some categories
df.T

**Checking the balance of the data**

In [ ]:
df.describe()

# I need to balance these 5 columns Rating, Votes, Meta Score, Year, Duration

In [ ]:
# I calculate the total number of rows in the DataFrame
# in order to check how many percentages of data was removed
# after the cleaning
original_rows = len(df)
original_rows

In [ ]:
# let's see the Rating distribution
sns.displot(df, x="Rating")

# we can notice that that the distribution is to more right side
# it would be good to cut it until 5 on the x-axis
# in order to make more balanced

In [ ]:
# filter out top 8%  (1 - 0.08 = 0.92) based on price to reduce amount of buildings
df = df.query("Rating > Rating.quantile(0.1) and Rating < Rating.quantile(0.95)")

In [ ]:
# now data is more balanced
sns.displot(df, x="Rating")

In [ ]:
# I checked that I lost about 17% of data because of this outliner removing
df.describe()

In [ ]:
sns.displot(df, x="Votes")
# we can see that the column "Votes" is disbalanced

In [ ]:
# let's remove everything that is up than 0.85
df = df.query("Votes < Votes.quantile(0.85)")

In [ ]:
# let's check the data plot
sns.displot(df, x="Votes")

# the daya still in not balanced but 
# we already we removed 15% of data

In [ ]:
# We still need to carry out of Meta Score, Year, Duration
sns.displot(df, x="Meta Score")

# Meta Score looks pretty well
# I will remove a little bit from bottom

In [ ]:
# calculate the cutoff value for the bottom 3%
cutoff_value = df['Meta Score'].quantile(0.03)

# filter out rows where the 'Meta Score' values are below the cutoff
df = df[df['Meta Score'] >= cutoff_value]

In [ ]:
# let's check the plot
sns.displot(df, x="Meta Score")

# it looks pretty balanced

In [ ]:
# we can see that there are many outliers in the old movies
class_counts = df['Year'].value_counts()
class_counts

In [ ]:
# and also there is a huge number of recent movies
sns.displot(df, x="Year")

In [ ]:
# I used quantiles to remove 5% from the bottom
df = df.query('Year >= Year.quantile(q=0.05)')

In [ ]:
# it is not balanced but we have less outliers
sns.displot(df, x="Year")

In [ ]:
# let's check the last column Duration
sns.displot(df, x="Duration")

In [ ]:
# Calculate the cutoff value for the top 3%
cutoff_value = df['Duration'].quantile(0.97)

# Filter out rows where the 'Meta Score' values are above the cutoff
df = df[df['Duration'] <= cutoff_value]

In [ ]:
# let's see
sns.displot(df, x="Duration")

# it is an almost perfect balance :)

In [ ]:
df.describe()

In [ ]:
# calculate the amount of rows in df after cleaning
total_rows_after_cleaning = len(df)

# Calculate the total percentage of data lost across all columns
total_percentage_lost = round(((original_rows - total_rows_after_cleaning) / original_rows) * 100, 3)
print("Total percentage of data lost:", total_percentage_lost)

**The train/test –split**

In [ ]:
# print the columns so we can copy the other 
# columns into the X later easily
df.columns

In [ ]:
# I define X and y -variables
# list here independent variables
X = df[['Votes', 'Meta Score', 'Year', 'Duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western',
       'PG Rating_13+', 'PG Rating_16+', 'PG Rating_18+', 'PG Rating_Approved',
       'PG Rating_G', 'PG Rating_GP', 'PG Rating_NC-17', 'PG Rating_PG',
       'PG Rating_PG-13', 'PG Rating_Passed', 'PG Rating_R', 'PG Rating_TV-14',
       'PG Rating_TV-G', 'PG Rating_TV-MA', 'PG Rating_TV-PG',
       'PG Rating_TV-Y7']]


# here we have only the target variable (dependent variable)
y = df['Rating']

In [ ]:
# create train and test sets of my dataset
# this examples reserves 20% for test data, 80% for training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

 **Train our Linear Regression model**

In [ ]:
# create an empty linear regression model
# and fit it with out data 
lm = LinearRegression()
lm.fit(X_train, y_train)

 **Error and performance metrics**

In [ ]:
# make test predictions with new data with our model
predictions = lm.predict(X_test)

In [ ]:
# these values follow a line = good predictions
# we basically compare the predicted values 
# to true test values and see the differences
plt.scatter(y_test, predictions)
plt.show()

In [ ]:
df.head()

** Create a tester row for metrics**

In [ ]:
# let's try with some new imaginary data
tester_row = {
    'Votes': 20000, 
    'Meta Score': 54.0, 
    'Year': 2020, 
    'Duration': 117, 
    'Action': 1,'Adventure': 1, 'Animation': 1, 'Biography': 0, 'Comedy': 0, 'Crime': 0, 'Documentary': 0,
       'Drama': 0, 'Family': 0, 'Fantasy': 0, 'History': 0, 'Horror': 0, 'Music': 0, 'Musical': 0,
       'Mystery': 0, 'Romance': 0, 'Sci-Fi': 0, 'Sport': 0, 'Thriller': 0, 'War': 0, 'Western': 0,
       
    'PG Rating_13+': 1, 'PG Rating_16+': 0, 'PG Rating_18+': 0, 'PG Rating_Approved': 0,
       'PG Rating_G': 0, 'PG Rating_GP': 0, 'PG Rating_NC-17': 0, 'PG Rating_PG': 0,
       'PG Rating_PG-13': 0, 'PG Rating_Passed': 0, 'PG Rating_R': 0, 'PG Rating_TV-14': 0,
       'PG Rating_TV-G': 0, 'PG Rating_TV-MA': 0, 'PG Rating_TV-PG': 0,
       'PG Rating_TV-Y7': 0
}

# convert to pandas-format
tester_row = pd.DataFrame([tester_row])

In [ ]:
# use our model to predict our tester_row data
result = lm.predict(tester_row)[0]

print()
print(f"Predicted rating for this movie: {round(float(result), 2)}")

In [ ]:
# Lower values indicate better performance - MAE, MSE, RMSE
# R-squared: 1 is perfect, 0 is worst outcome

# MAE - Mean average error
# The average of all errors
# Simple, but doesn't take large errors into account that much
print("MAE")
print(round(metrics.mean_absolute_error(y_test, predictions), 2))

# MSE - Mean square error
# Errors are computed in the power of 2 (squared) => larger mistakes are emphasized
print("\nMSE")
print(round(metrics.mean_squared_error(y_test, predictions), 2), "^2")

# RMSE - Root mean square error
# the most common error metric used in regression
# 0 = perfect RMSE-score, 1 = worst possible RMSE-score
print('\nRMSE:')
print(round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 2))

# R-squared. 0 = the model describes the dataset poorly
# 1 = model describes the dataset perfectly
print('\nR-squared:')
print(round(metrics.r2_score(y_test, predictions), 2))

In [ ]:
# if the prediction distribution are far from normal distribution
# then the model is not probably good enough
# distplot is deprecating in future pandas-version
# unfortunately, there's no exact alternative to do this plot at the moment
sns.distplot((y_test - predictions))
plt.show()
plt.close()

# we can see that there is a normal distribution 

**Emphasizing variables that should affect the prediction greatly**

In [ ]:
# instal the needed module
import dtale

In [ ]:
# let's inspect correlation matrix with dtale
# 1 = a perfect positive correlation
# -1 = a negative correlation
# 0 = no linear correlation between the variables
dtale.show(df)

# for more convienient usage there is an option in the right corner
# to open in a new tab after choosing "Correlations" in the Vizualizse button

# we can see that the table shows correlation between
# Rating - Meta Score = 0.53
# Rating - Votes = 0.37

# and there are some surprises, f.e.,
# Rating - Year = -0.28
# and also Duration does not plau a significant role
# considering this dtale table
# Rating - Duration = 0.18

In [ ]:
# By this dtale table I found that some columns 
# such as PG Rating_16+, PG Rating_18+, PG Rating_Approve and etc 
# have N/A values
# first of all, I though that maybe they contain NaN values
# I checked it
nan_values = df.isna().sum()
nan_values

# no, all NaN are removed

In [ ]:
# I assumed that it simply possible that these columns 
# have all zeros
# In this case the correlation coefficient cannot be computed 
# because the standard deviation of that variable is zero


# check if all values in a column are zeros
def check_column_only_zeros(column):
    return (column == 0).all()

# Check each column for only zeros
for column in df.columns:
    if check_column_only_zeros(df[column]):
        print(f"Column '{column}' contains only zeros.")

In [ ]:
# I can drop these columns
df = df.drop(columns=['PG Rating_16+', 'PG Rating_18+', 'PG Rating_Approved', 
                      'PG Rating_GP', 'PG Rating_Passed', 'PG Rating_TV-G'])

In [ ]:
# import the needed module
from sklearn.feature_selection import f_classif

# get the score rankings
f_values, _ = f_classif(X, y)

# Create a pandas Series for easier interpretation
feat_importances = pd.Series(f_values, index=X.columns)

# Plot the ANOVA F-value scores
plt.figure(figsize=(10, 8))  # the figure size
feat_importances.plot(kind='barh')
plt.xlabel('Significant value')
plt.ylabel('Columns')
plt.title('How statistically significant each variable is in the dataset')
plt.tight_layout()  # spacing to prevent overlapping
plt.show()


# Here we can see that for some unknown reasons
# the Drama column is importnant
# but as we saw earlier ones of the most significant 
# are Votes, Meta Score, Year, Duration

In [ ]:
# import needed modules
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression

# Assuming X is a feature matrix and y is a target variable
# Replace X and y with my actual data
selector = SelectKBest(score_func=f_regression, k=10)  # Select top 10 features
selector.fit(X, y)

# Get selected feature indices
selected_indices = selector.get_support(indices=True)

# Get names of selected features
selected_features = X.columns[selected_indices]

# Get F-scores of selected features
feature_scores = selector.scores_[selected_indices]

# Create DataFrame with feature names and scores
feature_scores_df = pd.DataFrame({
    'Feature': selected_features,
    'Score': feature_scores
})

# Sort DataFrame by score in descending order
feature_scores_df = feature_scores_df.sort_values(by='Score', ascending=False)

# Print DataFrame
print(feature_scores_df)


# for some reasaons (probably for same reasons :) )
# the Drama column has a significant effect on Rating
# Votes, Meta Score, Year, Duration

In [ ]:
# let's inspect Rating vs Drama since it's weird 
# when comparing to correlation matrix,  ANOVA F-value scores and SelectKBest
sns.boxplot(data=df, x="Rating", y="Drama")

# adjust the tick labels on the x-axis
# otherwise, they are too close to each other
plt.xticks(rotation=45)  # rotate the tick labels by 45 degrees for better readability
plt.tight_layout()  # adjust layout to prevent clipping of labels
plt.show()

# to be honest, I am not sure that there is any use from this plot
# let's try another one

In [ ]:
# we can try the regresion plot
sns.regplot(data=df, x='Rating', y='Drama')

# I think, that there could be a correlation:
# if a movie is drama -> more chances to have a better Rating

In [ ]:
# let's test this idea
# the first variant with choosing the drama genre
# let's try with some new imaginary data
tester_row = {
    'Votes': 20000, 
    'Meta Score': 54.0, 
    'Year': 2020, 
    'Duration': 117, 
    'Action': 0,'Adventure': 0, 'Animation': 1, 'Biography': 0, 'Comedy': 0, 'Crime': 0, 'Documentary': 0,
       'Drama': 1, 'Family': 0, 'Fantasy': 0, 'History': 0, 'Horror': 0, 'Music': 0, 'Musical': 0,
       'Mystery': 0, 'Romance': 0, 'Sci-Fi': 0, 'Sport': 0, 'Thriller': 0, 'War': 0, 'Western': 0,
       
    'PG Rating_13+': 1, 'PG Rating_16+': 0, 'PG Rating_18+': 0, 'PG Rating_Approved': 0,
       'PG Rating_G': 0, 'PG Rating_GP': 0, 'PG Rating_NC-17': 0, 'PG Rating_PG': 0,
       'PG Rating_PG-13': 0, 'PG Rating_Passed': 0, 'PG Rating_R': 0, 'PG Rating_TV-14': 0,
       'PG Rating_TV-G': 0, 'PG Rating_TV-MA': 0, 'PG Rating_TV-PG': 0,
       'PG Rating_TV-Y7': 0
}

# convert to pandas-format
tester_row = pd.DataFrame([tester_row])

# use our model to predict our tester_row data
result = lm.predict(tester_row)[0]

print()
print(f"Predicted rating for the movie with the Drama genre: {round(float(result), 2)}")

In [ ]:
# let's test this idea
# the first variant with choosing the drama genre
# let's try with some new imaginary data
tester_row = {
    'Votes': 20000, 
    'Meta Score': 54.0, 
    'Year': 2020, 
    'Duration': 117, 
    'Action': 0,'Adventure': 0, 'Animation': 1, 'Biography': 0, 'Comedy': 0, 'Crime': 0, 'Documentary': 0,
       'Drama': 0, 'Family': 0, 'Fantasy': 0, 'History': 0, 'Horror': 0, 'Music': 0, 'Musical': 0,
       'Mystery': 0, 'Romance': 0, 'Sci-Fi': 0, 'Sport': 0, 'Thriller': 0, 'War': 0, 'Western': 0,
       
    'PG Rating_13+': 1, 'PG Rating_16+': 0, 'PG Rating_18+': 0, 'PG Rating_Approved': 0,
       'PG Rating_G': 0, 'PG Rating_GP': 0, 'PG Rating_NC-17': 0, 'PG Rating_PG': 0,
       'PG Rating_PG-13': 0, 'PG Rating_Passed': 0, 'PG Rating_R': 0, 'PG Rating_TV-14': 0,
       'PG Rating_TV-G': 0, 'PG Rating_TV-MA': 0, 'PG Rating_TV-PG': 0,
       'PG Rating_TV-Y7': 0
}

# convert to pandas-format
tester_row = pd.DataFrame([tester_row])

# use our model to predict our tester_row data
result = lm.predict(tester_row)[0]

print()
print(f"Predicted rating for the movie WITHOUT the Drama genre: {round(float(result), 2)}")

In [ ]:
# we can see that the Drama actually affects on the Rating

**Some analysis**

<br>**Where could linear regression be useful in working life?** 
I think that movie directors or investors who want to check the potential rating of a movie before its creation or investment can use linear regression.

<br>**Was it easy or difficult to use?**
For me, it was easy to implement because I understand why we should take certain steps. Also, when I had questions, the teacher answered them for me.

<br>**Any ideas for optimizations?**
I believe that after conducting further analysis and gaining a better understanding of the significance of columns, I may decide to drop some of the PG-Rating and Genre columns in the future. Additionally, it is feasible to add more data since this rating list was updated until December 15, 2023, as the dataset was generated by web scraping the official IMDb site.

I also plan to incorporate more data through web scraping in future iterations. Furthermore, I think to create a simple user interface for my model, allowing users to input test values for the regression.